# PyTorch and Facial Keypoint Detection
<p style="text-align:justify"> Welcome to the first exercise of this CNN Tutorial! In this exercise we will build a
CNN for facial key point detection. The implementation will be in PyTorch. The exercise has been inspired by Kaggel challenge [[1]](#references_cell) and the solution to it by Daniel Nouri [[2]](#references_cell). The exercise has two parts - first, we will briefly learn about the required PyTorch for this CNN tutorial and then build a  simple CNN for facial key point detection. So let's move into PyTorch.

<span style="color:red">Note: Initially you will be guided through, being asked to read the code (except at one place where you have to fill the code) but a little later you will be coding!!!</span>

## PyTorch
<p style = "textalign:justify"> You can think of PyTorch as a replacement for numpy to use the power of GPU's and a deep learning package that provides maximum flexibility and speed. All of you must have used ndarrays (also called Tensors) in numpy. PyTorch allows you to use these Tensors on CPU or GPU through its routines.

In comparison to other packages like TensorFlow, Theano, Caffe where the deep network is built statically and so, a change in the network means rebuild it from the scratch, PyTorch allows change in the behaviour of the network dynamically with no lag or overhead. It is deeply integrated into Python in the sense that we can use it naturally like how we have been using numpy, scipy etc. To know more about what is PyTorch all about, look at [[3]](#references_cell).

Let me caution you that the attempt here is not to provide a complete tutorial in PyTorch. We will briefly cover the class *Variable* from the package torch.autograd, the class *Module* from the package *torch.nn* and the required material from  package *torch.optim*. For a complete set of tutorials you may refer to [[4]](#references_cell).

### Class *Variable* from *torch.autograd*
<p style = "textalign:justify"> The *Variable* class is a wrapper over torch Tensors (nd arrays in torch) that supports nearly all operations defined on tensors. PyTorch requires that the input tensor to be forward propagated has to be wrapped in a *Variable*. This facilitates automatic back propagation by simply calling the method *backward()* in the *Variable* class. Let us look at an example. Say, we want to compute $y = x^{2} + 3x -7; \, \, z = 4/y$ and back propagate the gradients at $x = 1$ i.e. we want to compute $\frac{dz}{dx}$ at  $x = 1$. On paper the result will be *-2.22* respectively. Let us do this in PyTorch.

In [46]:
#First go through and then run this cell 
import torch
from torch.autograd import Variable

x = Variable(torch.Tensor([1]), requires_grad = True) # x is a 1 dim tensor with value 1 wrapped by 
                                                      #the Variable class; we want gradient w.r.to x
print("x:", x.data) # x.data is the tensor with value 1. x is the Variable wrapper over the tensor x.data 

y = x ** 2 + 3 * x - 7 # first step in forward direction; since x is of type Variable, y also is.
z = 4  / y # second step in forward direction; z is also of type Variable

z.backward() # call the backward() method in Variable class on the output variabe z for automatic back propagation

print("dz/dx {}" .format(x.grad.data)) # x.grad is the Variable that contains gradient of output with repsect to x
                                  # x.grad.data is the tensor wrapped by Variable x.grad

# Note x could be any nd tensor. Say we want to compute y = Ax and z = yT A y where  A is a 2 x 2 matrix of 1's
# and x is 2 x 1 vector of ones 
# Uncomment the print statements below to see x, A and dz/dx which should be 2 x 1 tensor with value 16
x = Variable(torch.ones(2, 1), requires_grad = True)
A = Variable(torch.ones(2, 2), requires_grad = False)
#print("x:", x.data)
#print("A:", A.data)
y = torch.matmul(A, x) # torch.ones(2, 2) is a 2 x 2 tensor of ones. x is Variable. x.data is
                                           # corresponding tensor 
z = torch.matmul(torch.transpose(y, 0, 1), torch.matmul(A, y))

z.backward() 

#print("dz/dx {}" .format(x.grad.data))

<span style="color:magenta">
Answer the question given in the comment line of the following cell.
</span>

In [44]:
# Write PyTorch code in this cell below the comment lines to compute and print gradient of output 
# with respect to x where output = ||A x||^2. ||.|| denotes 2 norm, 
# ^ 2 denotes raised to power 2.
# A is the matrix [1   2
#                  0   -1] and x is the vector [1 -1]T, T stands for transpose.
# use torch.norm() to computer norm. see http://pytorch.org/docs/master/torch.html?highlight=torch%20norm#torch.norm
# Creation of x is already done for you. Complete the rest.
# Expected output is [-2 -6]T.
x = Variable(torch.Tensor([[1], [-1]]), requires_grad = True)
#START CODE HERE (Approx 3 lines)



#END CODE HERE
print("d(output)/dx {}" .format(x.grad.data))


So, in summary, an identifier *x* of type Variable is a wrapper over a corresponding tensor. This tensor is available in .data attribute of Variable i.e *x.data*. To get gradient of an identifier y dependant on this tensor, use .grad attribute i.e *x.grad.data*. Note that the grad attribute of Variable class is itself of type Variable whose data is in .grad.data

Forward propagation leads to an output variable, say *output*. To do the back propagation, simply call *output.backward()*

### The Class *Module* from the package *torch.nn* 
<p style = "textalign:justify"> This class is the base class for all modules containing various layers including linear layers, convolutional layers, recurrent layers, pooling layers, non-linear activation layers etc for building the deep network. It also has a method called *forward()* that takes input, forward propagates through the layers defined in the module and computes the output. The layers themselves are child classes of *Module* with their own *forward()* method. To build our own model, we need to define a class that inherits from *Module* and also override the *forward()* method. Let us build a simple two layered fully connected (FC) network with following architecture:
- i/p size : 9216
- num of hidden nodes in 1st hidden layer: 100
- num of hidden nodes in 2nd hidden layer: 30
- between hidden layer 1 and 2, a ReLU activation is done

Here is the class *FcModel* that defines this model. You may look at 
[nn.Linear](http://pytorch.org/docs/master/nn.html#linear) and 
[nn.ReLU](http://pytorch.org/docs/master/nn.html#relu) for defining the hidden and activation layers.

In [66]:
import torch.nn as nn

class FcModel(nn.Module): # our class inherits from Module class
    
    # init method instaniates the layers of our model
    def __init__(self):   
        super().__init__() # call the parents init method
        
        self.fc1 = nn.Linear(9216, 100) # instantiate the first hidden layer. Linear class is a child of Module. 
                                        # When it is instantiated, weights and biases are automatically 
                                        # initialized to random.
                                        # fc1.weight is of type Variable. fc1.weight.data are the weights
                                        # fc1.weight.grad.data are the gradients of final o/p w.r.to weights
                        
        self.fc2 = nn.Linear(100, 30) # instantiate the second hidden layer. This has its own weights and biases 
                                      # as described above
        
        self.relu = nn.ReLU() # instantiate the non-linear ReLU activation layer. This is a child of Module
                              # There are no parameters associated with this layer
    
    # define the forward method
    def forward(self, x): # x is the input; In PyTorch, for 2d images with multiple channels, i/p tensor for 
                          # linear layer should be of the form mini_batch_size x num_of_ activations or pixels
                          # and i/p tensor for convolutional layer should be of the form 
                          # mini_batch_size x num_of_channels x height x width. Keep this in mind.            
                          # We will require this info later.
            
            # Since each layer has its own forward() method, we just need to call them in the required order here. 
            # For eg, to call forward() method associated with 1st hidden layer, we can either call fc1.forward(x)
            # or fc1(x). We will always follow the later convention.
            
            x = self.fc1(x) # call the forward method of 1st hidden layer
            x = self.relu(x) # call the forward method of relu activation layer
            x = self.fc2(x) # call the forward method of 2nd hidden layer
            return x
    # That's it!! We have built the model. 
    # Now we have to define a loss function to compute loss,
    # set the required optimizer and call backward() to propagate gradients backward, and update the weights in 
    # an iterative fashion
            

In [67]:
# Before we deal with loss, optimizer etc. let's instantiate the FcModel class
fc_model = FcModel()

# Have a look at the following two lines but do not uncomment as we will be working with CPU 
#if torch.cuda.is_available(): # if cuda is available, our model is ported to GPU. So simple in PyTorch!!!
#    fc_model.cuda()

<p style = "textalign:justify"> A loss function takes (output, ground_truth) pair and computes the deviation of output from ground_truth. *torch.nn* package has several loss functions. We will use mean-squared loss for this exercise. You may look at [nn.MSELoss](http://pytorch.org/docs/master/nn.html#mseloss). We will set our optimizer for updating weights to be Stochastic Gradient Descent (SGD). This is available in *torch.optim* package. You may look at [SGD](http://pytorch.org/docs/master/optim.html#algorithms). Let's write the code.

In [54]:
import torch.optim

loss_criterion = nn.MSELoss(size_average = True) # instantiate loss. size_average = True will average loss over
                                                 # samples in each mini-batch. MSELoss is a child class of Module.
                                                 # It has its own forward function which we will call below
        
optimizer = torch.optim.SGD(fc_model.parameters(), lr = 0.01, momentum = 0.9)
                                                 # optimizers requires as first argument parameters to be updated.
                                                 # This is available in fc_model.parameters().
                                                 # parameters() is a method in Module class that returns an iterator
                                                 # over all parameters of the module. Our module here is fc_model
                                                 # second argument lr is learning rate
                                                 # third argument is momentum
                                                 # All the optimizers are children of torch.optim.Optimizer class
                                                 # This class has a method called step() which are overidden by each
                                                 # of its children. By calling this method parameters including
                                                 # weights and biases are automatically updated. We will call this
                                                 # method while training

## Facial Keypoint Detection
<p style = "textalign:justify"> We are about to train the model now. But what about the dataset?? You might have wondered why did we choose i/p size to be 9216, o/p size to be 30, MSELoss etc. The reason is because we were preparing you to build a model to automatically detect keypoints in a face. Let us discuss this problem now. Given a set of faces marked with $(x, y)$ coordinates of facial keypoints, say, left_eye left corner, left_eye right corner, right_eye left corner, right_eye right corner etc - can we build and train a model that if given a seen or unseen face will automatically detect the coordinates of the facial keypoints. See Figure 1 for an example of images with keypoints marked. 

<img src="images/facial_keypoints.png" style="width:350px;height:150px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **Faces with keypoints** </center></caption>
<br>

Keypoints across all faces for training are to be ordered consistently. We will be using the facial keypoints dataset from Kaggle challenge [[1]](#references_cell). The training set consists of 7,049 96x96 gray-scale images along with ground_truth (x, y) coordinates of 15 keypoints (totalling to 30 values) as shown in Figure 1. There are few more issues associated with this dataset like missing data which we will not be discussing here. We will only use a subset of this data for our experiment below. In fact we will not be covering details related to aspects of loading datasets in most of the CNN tutorial for the want of time. We have already written functions that will do it. You just need to run the cells that call those functions. 

Now, let us come back to the questions raised in the beginning of this cell. It should be clear that i/p size is 9216 (96 x 96) as linear layer (fully connected layer) requires the 2d data to be flattened to 1d. Also o/p size is 30 because the model has to predict 30 values. We are using MSELoss because we have a regression problem at hand.

So, let us load the dataset.

In [75]:
# Run this cell to load train and valid data. It will take around 30 seconds
from facial_keypoints_utils import *
dataloaders, dataset_sizes = load('./data/training.csv', test_split = 0.2) # dataloaders are iterators that supply
                                  # a mini_batch of data to the model. We have set the mini_batch_size to 64.
                                  # So the i/p size for FcModel will be 64 x 9216
                                  # Further we have split the data into training data (80%) and 
                                  # validation data (20%) 
                                  # So the dataloaders here is a dictionary that has two keys 'train' and 'valid' 
                                  # each of which corresponds to an iterator as described above
print(dataset_sizes)

Now, let us write the train_model function. There are certain points to keep in mind.
- During training phase, model should be set to train mode and *backward()* should be called
- During validation pahse, model should be set to *eval()* mode. *backward()* should not be called
- inputs and ground truth should be wrapped in  Variables respectively before being passed to *forward()* methods     of model and loss_criterion respectively
- Gradients should be explicitly zeroed in every epoch so that the previous epoch accumulated gradients do not       interfere in current epoch
- *step()* method of optimizer should be called after back propagation to update parameters of the model


In [29]:
# Have a look at the train_model function, especially the lines enclosed between #*********#
# After having had the look, run this cell
import time

def train_model(model, dataloaders, dataset_sizes, loss_criterion, optimizer):    

    since = time.time()
    valid_loss_history = []
    train_loss_history = [] 
    num_epochs = model.num_epochs
    
    for epoch in range(num_epochs):

        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'valid']:
            #***************************************************************#
            if phase == 'train':
                model.train(True) # train phase; set model's mode to training 
                                       
            else:
                model.train(False) # validation phase; set model's mode to eval. 
                                   # You can also do this by saying model.eval()
            #***************************************************************#  
            
            running_loss = 0.0
            
            # Iterate over data. 
            for data in dataloaders[phase]:
                # get the inputs
                inputs = data['image'] # a batch of inputs. i.e a matrix of size 64 x 9216
                gt = data['keypoints'] # corresponding ground truth i.e a matrix of size 64 x 30
                
                #*************************************************************#
                inputs, gt = Variable(inputs), Variable(gt) # wrap inputs and ground_truth in Variables
                
                # Have a look at the following if..else but do not uncomment as we will be working with cpu
                """
                if torch.cuda.is_available():
                    inputs, gt = Variable(inputs.cuda()), Variable(gt.cuda())
                                              
                                              # we have already ported model's parameters to GPU if available
                                              # Here we port inputs and ground_truth.
                                              # So everything required ported to GPU if available
                                              # PyTorch will run in GPU if available. So simple to port to GPU!!!
                else:
                    inputs, gt = Variable(inputs), Variable(gt)"""
                #*************************************************************#
                
                #*************************************************************#
                optimizer.zero_grad() # zero all the gradients in the beginning of every epoch for
                                      # reasons described above
                #*************************************************************#
                
                #*************************************************************#
                outputs = model(inputs) # forward propagate inputs. Note this is same as calling 
                                        # model.forward(inputs)
                #*************************************************************#
                
                #*************************************************************#
                loss = loss_criterion(outputs, gt) # compute the loss by calling forward method of loss class
                #*************************************************************#
                
                #*************************************************************# 
                if phase == 'train':
                    loss.backward()  # Having computed loss, backpropagate - but only in train mode
                    optimizer.step() # Gradients are ready!! Update the parameters by 
                                     # calling step() method of the optimizer
                #*************************************************************#
                
                # statistics
                running_loss += loss.data[0]

            epoch_loss = running_loss / dataset_sizes[phase]
            if phase == 'train':
                train_loss_history.append(epoch_loss)
            else:
                valid_loss_history.append(epoch_loss)
            print('{} Loss: {:.8f}'.format(
                    phase, epoch_loss))
            if phase == 'valid':
                print('Train Loss / Valid Loss: {:.6f}'.format(
                    train_loss_history[-1] / valid_loss_history[-1]))
                
    loss_history = (train_loss_history, valid_loss_history)
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
                time_elapsed // 60, time_elapsed % 60))    
    return 

In [70]:
# Let's train the model for two epochs
# Run this cell
fc_model.num_epochs = 2
train_model(fc_model, dataloaders, dataset_sizes, loss_criterion, optimizer)

<p style = "textalign:justify"> We trained only for two epochs because of time crunch. But we have pretrained this model for 400 epochs earlier with the only difference being that weights were intialized using Xavier method [5] instead of random intitalization. Training should complete in less than 5 minutes in Intel core i5 - 3.1 GHz CPU. We will load that model and test on few faces.

In [44]:
# Run this cell
from facial_keypoints_utils import *
%matplotlib inline
%load_ext autoreload
%autoreload 2
# load test set and predict results
test_fc_model()



<p style = "textalign:justify"> Not at all a bad set of results for a simple linear model!! There are quite a few faces where the nose keypoint is clearly misaligned, especially when face is not frontal. Let us try to improvise this using a deep convolutional network. Now you will be in action as you will be coding a bit. The model we are going to build is as follows:

i/p-->conv layer-->ReLU-->maxpool-->conv layer-->ReLU-->maxpool-->conv layer-->ReLU-->maxpool-->linear-->ReLU
-->linear-->ReLU-->Linear

The details are as follows:
<span style="color:brown">
<br>
i/p: 1 x 96 x 96
<br>
conv1: in_channels = 1, num_of_filters also called outchannels = 32, kernel_size = (3, 3), stride = (1, 1), no            padding
<br>
pool1: max pooling, kernel_size = (2, 2), no padding
<br>
conv2: in_channels = 32, outchannels = 64, kernel_size = (2, 2), stride = (1, 1), no padding
<br>
pool2: max pooling, kernel_size = (2, 2), no padding
<br>
conv3: in_channels = 64, outchannels = 128, kernel_size = (2, 2), stride = (1, 1), no padding
<br>
pool3: max pooling, kernel_size = (2, 2), no padding
<br>
fc1: in_features = 128, out_features = 500
<br>
fc2: in_features = 500, out_features = 500
<br>
fc3: in_features = 500, out_features = 30
</span>
<br>
<br>
**Dimensions through the model:**
<br>
<br>
1x96x96 $\xrightarrow[\text{relu}]{\text{conv +}}$ 32x94x94 $\xrightarrow[\text{}]{\text{pool}}$ 32x47x47 $\xrightarrow[\text{relu}]{\text{conv +}}$ 64x46x46 $\xrightarrow[\text{}]{\text{pool}}$ 64x23x23$\xrightarrow[\text{relu}]{\text{conv +}}$ 128x22x22 $\xrightarrow[\text{}]{\text{pool}}$ 128x11x11 = 15488$\xrightarrow[\text{relu}]{\text{fc +}}$ 500 $\xrightarrow[\text{relu}]{\text{fc +}}$ 500$\xrightarrow[\text{}]{\text{fc}}$ 30

We will build a class called ConvModel and also define its *forward()* method. 
You may take help from [nn.Conv2d](http://pytorch.org/docs/master/nn.html#conv2d), 
[nn.ReLU](http://pytorch.org/docs/master/nn.html#relu), 
[nn.MaxPool2d](http://pytorch.org/docs/master/nn.html#maxpool2d) and 
[nn.Linear](http://pytorch.org/docs/master/nn.html#linear).

We have filled the first conv, relu and pool layers to give an idea. **You need to complete the rest by replacing *None* in the rhs of the following cell by your code**. Also populate the *forward()* method with correct code. In the *forward()* method, note that before moving to linear layer the i/p from 3rd pooling layer must be flattened. You may take help from 
[torch.Tensor.view](http://pytorch.org/docs/master/tensors.html?highlight=torch%20view#torch.Tensor.view)

 You may look at our first model built a little while ago for help. All the best for you to complete the exercise successfully!!

In [32]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim

class ConvModel(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = (3, 3), stride = (1, 1))
        self.relu1_c = nn.ReLU()
        self.pool1 = nn.MaxPool2d(2, stride = (2, 2)) 
        
        self.conv2 = None
        self.relu2_c = None
        self.pool2 = None
        self.conv3 = None
        self.relu3_c = None
        self.pool3 = None
        self.fc1 = None
        self.relu1_f = None
        self.fc2 = None
        self.relu2_f = None
        self.fc3 = None
        
    def forward(self, x):
        # remove the pass statement and populate this function with correct code
        # make sure at the end there is return statement that returns o/p of forward propagation
         pass  
       
        

In [24]:
# Run this cell to test your code and see if you get expected result
torch.manual_seed(23)
cm = ConvModel()
ip = Variable(torch.randn(1, 1, 96, 96))
op = cm(ip)
print(op)

<b> Expected Output:</b>
<br>
<span style="color:green">
<br>
Variable containing:
<br>
Columns 0 to 9 
<br>
1.00000e-02 \*
<br>
  2.8687 -3.2292 -1.3788 -2.7659 -5.4250  3.9658 -3.2263  5.8739 -0.6396  1.7851
<br>
Columns 10 to 19 
<br>
1.00000e-02 \*
<br>
 -2.5846  0.6233  0.6157 -2.0175 -3.7972  1.9028  1.6063 -7.6084  5.1107  0.5540
<br>
Columns 20 to 29 
<br>
1.00000e-02 \*
<br>
 -3.5491  5.2544  7.5331 -6.9017  3.7332 -3.5497 -6.7909  3.9685 -3.2628  0.2684
<br>
[torch.FloatTensor of size 1x30]</span>

In [36]:
# Run this cell to load the data

from facial_keypoints_utils import *

dataloaders, dataset_sizes = load('./data/training.csv', test_split = 0.2, conv = True)

<p style = "testalign:justify"> Now you will instantiate ConvModel, set num_epochs to 1, port it to GPU if available, instantiate loss criterion to MSELoss with size_average = True, instantiate the optimizer to SGD with learning rate = 0.01 and momentum = 0.9 respectively and call the train_model function defined above. Training for one epoch will take around 20 seconds. Complete the code in the cell below by replacing None statements on rhs by your code.

In [39]:
# complete the code and run the cell and check if you get the expected output
torch.manual_seed(23) #do not change this line

# Instantiate ConvModel
conv_model = None
#set num_epochs to 1
conv_model.num_epochs = None

# Write code below (maximum of two lines) to port conv_model to GPU if available




# Instantiate loss_criterion to MSELoss with size_average = True
loss_criterion = None
# Instantiate optimizer to SGD with learning rate 0.01 and momentum 0.9
optimizer = None

# Call train_model function below (1 line of code). It is defined in a earlier cell 


<b> Expected Output:</b>
<br>
<span style="color:green">
<br>
Epoch 0/0
<br>
\----------
<br>
train Loss: 0.00204450
<br>
valid Loss: 0.00140347
<br>
Train Loss / Valid Loss: 1.456744
<br>
Training complete in 0m 21s

<p style = "textalign:justify">Now, lets predict the performance of the model. As before we have earlier trained this model but with a triple of changes from what you had done - (i) initialization is based on Xavier method [5], (ii) images randomly horizontally flipped during training (data augumentation) and (iii) learning rate exponentially decayed. Data augumentation adds value though writing code for incorporating random horizontal flipping is not at all difficult. We trained for 1000 iterations in Intel core i5 - 3.1 GHz CPU and it took around 35 minutes. Lets load this pre-trained model and see how the convolutional network performs on the previous set of test faces.

In [42]:
#Run this cell to test ConvModel and predict keypoints
%matplotlib inline
model = ConvModel()
predict_plot_conv(model, './data/test.csv', 'facial_keypoints_model4.pth')

<p style = "textalign:justify"> <span style="color:purple">Surely an improvisation over linear model. But for one image, the convModel has done very well in almost all the images. We are sure that you must have gained some experience in CNN and Pytorch. So, welcome to the world of deep learning. Congratulations!! Could have done much more in this one exercise but for a one day tutorial with plenty of agenda ahead, you must have already got saturated with basic CNN already. Lets go ahead to see a lot of other variants.</span>

<a id='references_cell'></a>

### References 

1. https://www.kaggle.com/c/facial-keypoints-detection

2. http://danielnouri.org/notes/2014/12/17/using-convolutional-neural-nets-to-detect-facial-keypoints-tutorial/

3. http://pytorch.org/about/

4. http://pytorch.org/tutorials/

5. [Glorot, X. & Bengio, Y. - Understanding the difficulty of training deep feedforward neural networks (2010)](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.207.2059&rep=rep1&type=pdf)